In [1]:
# Import necessary libraries
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from os import listdir
import re
import math
from sklearn.metrics import confusion_matrix

In [293]:
FILEPATH = 'ece661_pics\\hw8_image\\'

In [764]:
class Image():
    ''' 
        Class for storing images.
    '''
    
    def __init__(self, name):
        self.name = name    
        self.image = None  
        self.feature = None

    def load(self):
        self.image = cv.imread(self.name)
        self.image_gray = cv.cvtColor(self.image, cv.COLOR_BGR2GRAY)

    def show(self):
        if self.image is None:
            self.load()
        plt.imshow(cv.cvtColor(self.image, cv.COLOR_BGR2RGB))

    def compute_gram_matrix(self, filters):
        ''' 
            Compute Gram Matrix and 
            return only lower triangle part.
        '''
        
        if self.image is None:
            self.load()

        img = self.image_gray
        k = 16
        C = filters.shape[2]
        vectors = np.empty((k**2, C))
        for c in range(C):
            kernel = filters[:, :, c]
            # Convolve with each filter
            convolved = cv.filter2D(img, -1, kernel)
            # Down scale to 16 x 16
            down_scale = cv.resize(convolved, (k, k))
            # vectorize to 256 element vector
            vectors[:, c] = down_scale.ravel()

        feature = []
        for c in range(C):
            for i in range(c):
                # Inner product of each pair
                feature.append(np.dot(vectors[:, c].T, vectors[:, i]))

        self.feature = np.array(feature)

        return self.feature


In [765]:
def load_image(img_classes):
    '''
        Load images from database.
    '''

    trainning_set = {}
    testing_set = {}
    valid_set = {}
    
    for img_class in img_classes:
        trainning_set[img_class] = []
        testing_set[img_class] = []
        valid_set[img_class] = []
        # Load training sets
        directory = f'{FILEPATH}training'
        imgs = listdir(directory)
        imgs_in_class = [re.findall(f'{img_class}.*', img) for img in imgs]
        for img in imgs_in_class:
            if img:
                full_path = f'{directory}\\{img[0]}'
                trainning_set[img_class].append(Image(full_path))

        # Load testing set
        directory = f'{FILEPATH}testing'
        imgs = listdir(directory)
        imgs_in_class = [re.findall(f'{img_class}.*', img) for img in imgs]
        for img in imgs_in_class:
            if img:
                full_path = f'{directory}\\{img[0]}'
                testing_set[img_class].append(Image(full_path))

        # Create validation list
        total_imgs = len(trainning_set[img_class])
        valiation_ratio = 0.25
        for i in range(int(valiation_ratio * total_imgs)):
            idx = np.random.randint(0, total_imgs-i)
            valid_set[img_class].append(trainning_set[img_class].pop(idx))

    return trainning_set, testing_set, valid_set

In [841]:
img_classes = ['rain', 'cloudy', 'shine', 'sunrise']
[imgs_train, imgs_test, imgs_valid] = load_image(img_classes)

In [979]:
def create_filter(C):
    '''
        Create 3 x 3 x C filter which vaule range from [-1, 1]
        and the sum is 0.
    '''
    m = 3
    filters = np.empty((m, m, C))
    for c in range(C):
        sample = np.random.rand(m**2)
        sample -= sample.mean()
        sample /= np.abs(sample).max()
        filters[:, :, c] = sample.reshape((m ,m))

    return filters

In [982]:
filters = create_filter(10)
feature_size = imgs_train['rain'][0].compute_gram_matrix(filters).shape[0]

In [983]:
# Create training dataset
train_data = []
labels = []
for i, img_class in enumerate(img_classes):
    for img in imgs_train[img_class]:
        labels.append(i)
        train_data.append(img.compute_gram_matrix(filters))

labels = np.array(labels)
train_data = np.array(train_data).astype(np.float32)

In [984]:
# SVM parameters
svm = cv.ml.SVM_create()
svm.setType(cv.ml.SVM_C_SVC)
svm.setKernel(cv.ml.SVM_LINEAR)
svm.setTermCriteria((cv.TERM_CRITERIA_MAX_ITER, 100, 1e-6))
svm.train(train_data, cv.ml.ROW_SAMPLE, labels);

In [985]:
# Create validation  dataset
valid_data = []
true_labels = []
for i, img_class in enumerate(img_classes):
    for img in imgs_valid[img_class]:
        true_labels.append(i)
        valid_data.append(img.compute_gram_matrix(filters))

true_labels = np.array(true_labels)
valid_data = np.array(valid_data).astype(np.float32)
response = svm.predict(valid_data)[1]

validation = []
for i in range(len(response)):
    validation.append(response[i][0] == true_labels[i])

validation = np.array(validation, dtype=np.uint8)
print(f'Validation accuracy = {validation.mean()*100:.2f} %')
confusion_matrix(true_labels, response[:, 0])

Validation accuracy = 35.32 %


array([[15, 24,  7,  5],
       [ 2, 11,  6, 53],
       [10, 14,  9, 27],
       [ 6, 19,  1, 60]], dtype=int64)

In [986]:
# Create testing dataset
test_data = []
true_labels = []
for i, img_class in enumerate(img_classes):
    for img in imgs_test[img_class]:
        true_labels.append(i)
        test_data.append(img.compute_gram_matrix(filters))

true_labels = np.array(true_labels)
test_data = np.array(test_data).astype(np.float32)
response = svm.predict(test_data)[1]

result = []
for i in range(len(response)):
    # print(f' Truth {true_labels[i]} Predicted {response[i][0]} {response[i][0] == true_labels[i]}')
    result.append(response[i][0] == true_labels[i])

result = np.array(result, dtype=np.uint8)
print(f'Testing accuracy = {result.mean()*100:.2f} %')
confusion_matrix(true_labels, response[:, 0])

Testing accuracy = 35.00 %


array([[5, 1, 2, 2],
       [0, 3, 2, 5],
       [1, 3, 0, 6],
       [1, 2, 1, 6]], dtype=int64)

In [987]:
# Save model
svm.save(f'{FILEPATH}model.xml')

In [988]:
np.set_printoptions(precision=3)
for f in range(filters.shape[2]):
    print(filters[:, :, f])

[[ 0.53  -0.527 -0.948]
 [ 0.241  0.633  1.   ]
 [-0.875  0.615 -0.669]]
[[-0.638  0.233  0.586]
 [ 0.823 -0.533 -0.674]
 [-0.077  1.    -0.72 ]]
[[-0.315  0.032  0.446]
 [-0.446 -0.322 -0.199]
 [ 0.403  1.    -0.598]]
[[-0.279  1.    -0.405]
 [ 0.191 -0.133  0.86 ]
 [-0.193 -0.401 -0.639]]
[[ 0.75   0.348 -0.833]
 [ 0.09  -0.653 -0.714]
 [-0.442  0.454  1.   ]]
[[-0.985 -0.916  0.787]
 [ 0.934  0.975  0.76 ]
 [-0.558 -1.     0.004]]
[[ 0.246  0.472  1.   ]
 [-0.824  0.839  0.092]
 [-0.935 -0.598 -0.291]]
[[-0.483 -0.881 -0.285]
 [ 0.463  0.707 -0.024]
 [ 0.326 -0.822  1.   ]]
[[-0.361 -0.54   0.466]
 [ 1.     0.009 -0.477]
 [-0.264 -0.595  0.762]]
[[ 0.248 -0.518  0.246]
 [ 0.643 -0.122 -0.221]
 [ 0.342  0.382 -1.   ]]
